<h1 style="text-align: center;">Tim "Poslednji ispit" - University of Kragujevac</h1>

<p style="text-align: center;">
  <img src="logopmfy.png" alt="University of Kragujevac Logo" width="200" height="200">
</p>

<h2 style="text-align: center;">Team Members</h2>

<ul style="list-style-type: none; text-align: center;">
  <li><strong>Vuk Lazović</strong></li>
  <li><strong>Sara Velimirović</strong></li>
  <li><strong>Mihajlo Janković</strong></li>
</ul>


# Predstavljanje problema i motivacija

Predviđanje cena automobila ima značajnu vrednost kako za kupce, tako i za prodavce. Sa stanovišta kupaca, tačna predviđanja cena mogu im pomoći da donesu bolje informisane odluke, izbegnu preplate i identifikuju najbolje ponude. Prodavci, s druge strane, mogu koristiti ova predviđanja kako bi optimizovali svoje cene, postavili konkurentne cene i povećali šanse za bržu prodaju. Takođe, predviđanje cena može pomoći i investitorima i analitičarima u prepoznavanju tržišnih trendova i prilika za ulaganje. Odabrali smo ovaj problem zbog njegove praktične primene i mogućnosti da razvijemo modele mašinskog učenja koji mogu biti korisni u stvarnom svetu. Ovaj projekat pruža priliku da se analiziraju faktori koji najviše utiču na cenu vozila, kao što su godine proizvodnje, marka, model, kilometraža i stanje, čime se može doprineti boljem razumevanju tržišta polovnih automobila.

## Učitavanje podataka

In [1]:
pip install -r requirements.txt

  You can safely remove it manually.



  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Install

U ovom koraku proveravamo da li je dataset, odnosno fajl "vehicles.csv", već preuzet i nalazi se u radnom direktorijumu. Funkcija *check_dataset* pretražuje sve fajlove u trenutnom direktorijumu i proverava da li se fajl sa ovim imenom već nalazi tu. Ako fajl nije pronađen, pokreće se funkcija *download*, koja preuzima dataset sa interneta. Preuzimanje se obavlja u **segmentima (chunkovima)** kako bi proces bio efikasniji i omogućen je napredak preuzimanja koristeći *tqdm* za prikazivanje napretka. Ako je fajl već preuzet, korisnik će biti obavešten da je dataset već dostupan i preuzimanje se neće ponavljati. Ovaj korak osigurava da uvek radimo sa potrebnim podacima, a da se izbegne nepotrebno preuzimanje ako je fajl već dostupan.

In [2]:
import requests,os
from tqdm import tqdm


def check_dataset():
    files = os.listdir('.')
    flag=1
    for file in files:
        if("vehicles.csv"==file):
            flag=0
    return flag

def download():
    print("Downloading dataset.....")
    url = "https://www.dropbox.com/scl/fi/hiod02ra6fa1d5f5q7bmd/vehicles.csv?rlkey=ein4k3paqkw0ashh8njtyg6ed&st=gvyd3ohd&dl=1"
    response = requests.get(url, stream=True)
    
    # Get the total file size
    total_size = int(response.headers.get('content-length', 0))
    
    with open("vehicles.csv", mode="wb") as file:
        for chunk in tqdm(response.iter_content(chunk_size=10 * 1024), total=total_size//(10*1024), unit='KB'):
            file.write(chunk)

    response = requests.get(url, stream=True)


if(check_dataset()):
    download()
else:
    print("Dataset already downloaded")

141402KB [02:05, 1123.30KB/s]                            


## Pregled podataka

Dimenzije podataka: (426880, 26)

In [5]:
num_rows, num_cols = df.shape
print(f"Broj vrsta: {num_rows}")
print(f"Broj kolona: {num_cols}\n")

Broj vrsta: 426880
Broj kolona: 26



Nakon što je dataset uspešno preuzet, sledeći korak je učitavanje podataka u naš radni prostor. Koristimo biblioteku **pandas** kako bismo učitali CSV fajl "vehicles.csv" i smestili podatke u *DataFrame* objekat pod nazivom df. *DataFrame* predstavlja strukturirani format podataka u obliku tabele, što nam omogućava lakšu manipulaciju, analizu i vizualizaciju podataka. Učitavanjem podataka u *DataFrame*, postavljamo osnovu za dalju analizu i pripremu podataka koja će biti neophodna za kreiranje modela za predviđanje cena automobila.

Predstavićemo dataset i objasniti svaku varijablu.

In [2]:
import pandas as pd
df=pd.read_csv("vehicles.csv")
df

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,7301591192,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,...,NaN,sedan,NaN,https://images.craigslist.org/00o0o_iiraFnHg8q...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:31-0600
426876,7301591187,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,...,NaN,sedan,red,https://images.craigslist.org/00x0x_15sbgnxCIS...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:29-0600
426877,7301591147,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,...,NaN,hatchback,white,https://images.craigslist.org/00L0L_farM7bxnxR...,Carvana is the safer way to buy a car During t...,NaN,wy,33.779214,-84.411811,2021-04-04T03:21:17-0600
426878,7301591140,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,...,NaN,sedan,silver,https://images.craigslist.org/00z0z_bKnIVGLkDT...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:11-0600


Predstavljanje kolona unutar dataset-a i objašnjenje značenja svake.

In [3]:
list(df.columns)

['id',
 'url',
 'region',
 'region_url',
 'price',
 'year',
 'manufacturer',
 'model',
 'condition',
 'cylinders',
 'fuel',
 'odometer',
 'title_status',
 'transmission',
 'VIN',
 'drive',
 'size',
 'type',
 'paint_color',
 'image_url',
 'description',
 'county',
 'state',
 'lat',
 'long',
 'posting_date']


1. **id** - Jedinstveni identifikator za svaki unos u datasetu.

2. **url** - URL adresa do originalnog Craigslist oglasa za vozilo.

3. **region** - Geografski region u kojem se vozilo prodaje, kao što je grad ili oblast.

4. **region_url** - URL adresa Craigslist stranice za određeni region.

5. **price** - Cena vozila navedena u oglasu. Ovo je ciljna promenljiva koju želimo da predviđamo.

6. **year** - Godina proizvodnje vozila.

7. **manufacturer** - Proizvođač vozila, kao što su Ford, Toyota, Honda, itd.

8. **model** - Specifičan model vozila, npr. Camry, F-150, Civic.

9. **condition** - Stanje vozila prema navodima prodavca, npr. new (novo), like new (kao novo), excellent (odlično), good (dobro), fair (zadovoljavajuće), salvage (oštećeno).

10. **cylinders** - Broj cilindara motora vozila, npr. 4 cylinders, 6 cylinders, 8 cylinders.

11. **fuel** - Tip goriva koje vozilo koristi, npr. gas (benzin), diesel (dizel), hybrid (hibrid), electric (električni), other (drugo).

12. **odometer** - Pređena kilometraža vozila (odometar) izražena u miljama.

13. **title_status** - Status vlasničkog lista (naslov vozila), npr. clean (čist), salvage (oštećen), rebuilt (restauriran), lien (teret), missing (nedostaje), parts only (samo za delove).

14. **transmission** - Vrsta menjača u vozilu, npr. automatic (automatski), manual (ručni), other (drugo).

15. **VIN** - Jedinstveni identifikacioni broj vozila (Vehicle Identification Number).

16. **drive** - Pogonska konfiguracija vozila, npr. 4wd (četiri točka), fwd (prednji pogon), rwd (zadnji pogon).

17. **size** - Veličina vozila, npr. compact (kompaktno), full-size (pune veličine), mid-size (srednje veličine).

18. **type** - Tip vozila, npr. sedan, SUV, truck, coupe, van, wagon.

19. **paint_color** - Boja vozila prema navodima prodavca.

20. **image_url** - URL adresa do slike vozila iz oglasa.

21. **description** - Tekstualni opis vozila iz oglasa.

22. **county** - Okrug u kojem se vozilo prodaje (može biti prazan u mnogim slučajevima).

23. **state** - Američka savezna država u kojoj se vozilo prodaje, npr. CA, TX, NY.

24. **lat** - Geografska širina lokacije vozila (latitude).

25. **long** - Geografska dužina lokacije vozila (longitude).

26. **posting_date** - Datum kada je oglas za vozilo postavljen na Craigslist.

Izvršićemo grubi pregled dataset-a na osnovnu povratne vrednosti funckije *info*.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

1. **Popunjenost podataka**:

Većina kolona ima visoku popunjenost, što znači da su podaci u velikoj meri dostupni za analizu. Kolone poput *id*, *url*, *region*, *price*, *state*, *image_url*, i *posting_date* imaju gotovo sve unose popunjene. <br>
Neke kolone imaju značajan broj nedostajućih vrednosti, što će zahtevati dodatnu obradu ili imputaciju pre nego što budu korišćene u modelima. Na primer, kolone *condition*, *cylinders*, *VIN*, *drive*, *size*, i *paint_color* imaju veliki broj nedostajućih podataka. <br>
Kolona *county* nema nijednu popunjenu vrednost, što ukazuje na to da bi mogla biti irelevantna za analizu i može se razmotriti za isključivanje.
<br>

2. **Tipovi podataka**:

Kolone poput *price* i *year* su numeričke (int64, float64), što je korisno za statističke analize i modeliranje. <br>
Većina drugih kolona su object tipa, što označava tekstualne podatke ili kategorije. Neke od ovih kolona, poput *manufacturer*, *model*, i *fuel*, mogu biti pretvorene u kategorije za efikasniju analizu. <br>
Geografske koordinate *(lat i long)* su u numeričkom formatu, što omogućava prostorne analize.
<br>

3. **Potencijalni izazovi**:

Nedostajući podaci u ključnim kolonama, poput *condition*, *cylinders*, i *drive*, mogu predstavljati izazov za modeliranje i zahtevaće posebnu pažnju, kao što je imputacija vrednosti ili uklanjanje nekompletnih unosa.
Kolone sa malim brojem popunjenih vrednosti, poput *county* i *size*, možda neće biti korisne za analizu i mogu se razmotriti za uklanjanje.

### Odabir kolona

Kolone koje nećemo uzimati u obzir jer nisu relevatne: <br>
- id
- url
- region_url
- VIN
- image_url
- description
- posting_date

In [13]:
df_relevant_columns = df.drop(columns=['id', 'url', 'region_url', 'VIN', 'image_url', 'description', 'posting_date'])

Takođe, u obzir nećemo uzeti sledeće kolone:
- region
- lat
- long <br>

zato što je dovoljna količina informacija obuhvaćena kolonom **state**.

In [14]:
df_relevant_columns.drop(columns=['region', 'lat', 'long'], inplace=True)

Takođe, manje relevantna kolona je i:
- paint_color <br>


In [18]:
df_relevant_columns.drop(columns=['paint_color'], inplace=True)

Pregled koliko procenata null vrednosti ima svaka kolona:

In [20]:
df_relevant_columns.isna().mean()

price           0.000000
year            0.002823
manufacturer    0.041337
model           0.012362
condition       0.407852
cylinders       0.416225
fuel            0.007058
odometer        0.010307
title_status    0.019308
transmission    0.005988
drive           0.305863
type            0.217527
state           0.000000
dtype: float64

Kolone koje sadrže više od 60% null vrednosti, nema smisla popunjavati vrednostima, zato što bi moglo da dođe do pristrasnosti i samim tim bi jako uticalo na tačnost modela. Te kolone ćemo ukloniti.

In [21]:
null_columns = df_relevant_columns.columns[df_relevant_columns.isnull().mean()>0.60]
df_relevant_columns.drop(columns = null_columns,axis=1,inplace=True)
list(df_relevant_columns.columns)

['price',
 'year',
 'manufacturer',
 'model',
 'condition',
 'cylinders',
 'fuel',
 'odometer',
 'title_status',
 'transmission',
 'drive',
 'type',
 'state']

## !!!! NAPOMENA !!!!!
**OVO TREBA DA VIDIMO SA BRANKOM ILI SA NEKIM ?? DA LI JE BOLJE DA OVO STO IMA IZNAD 30% NEDOSTAJUCIH, DA LI JE BOLJE DA OBRISEMO TU KOLONU ILI DA OBRISEMO REDOVE KAKO BISMO REDUKOVALI BROJ NEDOSTAJUCIH VREDNOSTI.** <BR>
**MISLIM DA NE POSTOJI NACIN DA POPUNIMO TE KOLONE, JER NE ZAVISE OD NEKE DRUGE KOLONE, A AKO BI RANDOM POPUNJAVALI MISLI DA BI PREVISE DOLAZILO DO GRESKE MODELA**

*ja sam ovde obrisala redove, da bih mogla dole da nastavim da radim, ali to cemo da vidimo*
## !!!!

**ovde fali popunjavanje NA vrednosti ili brisanje u zavisnosti od potreba**

In [26]:
df_relevant_columns.dropna(inplace=True)

In [27]:
df_relevant_columns.isna().mean()

price           0.0
year            0.0
manufacturer    0.0
model           0.0
condition       0.0
cylinders       0.0
fuel            0.0
odometer        0.0
title_status    0.0
transmission    0.0
drive           0.0
type            0.0
state           0.0
dtype: float64

## Sređivanje podataka

Kolone sa numeričkim podacima su **price**, **year**, **odometer**.

In [25]:
df_relevant_columns.dtypes

price             int32
year            float64
manufacturer     object
model            object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
drive            object
type             object
state            object
dtype: object

Da bismo redukovali iskorišćenost memorije prebacićemo float64 u int64.

In [28]:
df_relevant_columns['year'] = df_relevant_columns['year'].astype(int)
df_relevant_columns['odometer'] = df_relevant_columns['odometer'].astype(int)

---------------------------------------------

Detaljnija analiza svake kolone posebno.

In [8]:
import numpy as np 
import pandas as pd 
from scipy import stats

def analyze_dataframe(df, n, m):
    # Number of rows and columns
    # n - the number of most frequent values of each feature to be analyzed 
    # m - the number of characters of each value that will be displayed for
    #     each feature with the string data type, the remaining values 
    #     are replaced by "..."
    # Returns - text describing about each feature of a dataset df


    # Analyze each feature
    for feature in df.columns:
        # Check for unique non-missing values
        unique_non_na_values = df[feature].dropna().nunique()
        if unique_non_na_values == 0:
            print(f"Feature '{feature}' has no unique values - all are missing")
            print("\n")
            continue

        print(f"Feature: {feature}")

        # Data type of the feature
        dtype = df[feature].dtype
        print(f"Data type: {dtype}")

        # Number of unique values
        unique_values = df[feature].nunique()
        print(f"Number of unique values: {unique_values}")

        # Percentage of values that are np.nan, np.inf, -np.inf
        total_values = len(df[feature])
        nan_values = df[feature].isna().sum()

        if pd.api.types.is_numeric_dtype(df[feature]):
            inf_values = np.isinf(df[feature]).sum()
        else:
            inf_values = 0

        invalid_values = nan_values + inf_values
        invalid_percentage = (invalid_values / total_values) * 100
        print(f"Percentage of np.nan, np.inf, -np.inf: {invalid_percentage:.2f}%")

        # Top 12 most frequent values
        top_n_values = df[feature].value_counts().head(n)
        top_n_values_list = top_n_values.index.tolist()
        top_n_percentage = (top_n_values.sum() / total_values) * 100

        if dtype == 'object':
            top_n_values_list = [
                (str(val)[:m] + '...') if len(str(val)) > m else str(val) 
                for val in top_n_values_list
            ]

        print(f"Top {n} most frequent values ({top_n_percentage:.2f}% of all values):")
        print(top_n_values_list)

        # Additional analysis for numeric features
        if pd.api.types.is_numeric_dtype(df[feature]):
            mean_value = df[feature].mean()
            median_value = df[feature].median()
            variance_value = df[feature].var()
            std_dev_value = df[feature].std()
            quantile_25 = df[feature].quantile(0.25)
            quantile_75 = df[feature].quantile(0.75)
            min_value = df[feature].min()
            max_value = df[feature].max()

            print("Numeric characteristics:")
            print(f"Mean: {mean_value}")
            print(f"Median: {median_value}")
            print(f"Variance: {variance_value}")
            print(f"Standard deviation: {std_dev_value}")
            print(f"1st quartile (25%): {quantile_25}")
            print(f"3rd quartile (75%): {quantile_75}")
            print(f"Minimum value: {min_value}")
            print(f"Maximum value: {max_value}")

            # Check for normal distribution
            k2, p = stats.normaltest(df[feature].dropna())
            alpha = 1e-3
            if p < alpha:  # null hypothesis: x comes from a normal distribution
                print("Not normal distribution")
            else:
                print("Normal distribution")

        print("\n")

In [9]:
analyze_dataframe(df, 12, 20)

Feature: id
Data type: int64
Number of unique values: 426880
Percentage of np.nan, np.inf, -np.inf: 0.00%
Top 12 most frequent values (0.00% of all values):
[7222695916, 7313139418, 7313423023, 7313423324, 7313424533, 7313425823, 7313426990, 7313427132, 7313426423, 7313426503, 7313427934, 7313428330]
Numeric characteristics:
Mean: 7311486634.224333
Median: 7312620821.0
Variance: 20009253539795.926
Standard deviation: 4473170.412559299
1st quartile (25%): 7308143339.25
3rd quartile (75%): 7315253543.5
Minimum value: 7207408119
Maximum value: 7317101084
Not normal distribution


Feature: url
Data type: object
Number of unique values: 426880
Percentage of np.nan, np.inf, -np.inf: 0.00%
Top 12 most frequent values (0.00% of all values):
['https://prescott.cra...', 'https://nh.craigslis...', 'https://nh.craigslis...', 'https://nh.craigslis...', 'https://nh.craigslis...', 'https://nh.craigslis...', 'https://nh.craigslis...', 'https://nh.craigslis...', 'https://nh.craigslis...', 'https://nh.c

In [13]:
df['year'].unique()

array([  nan, 2014., 2010., 2020., 2017., 2013., 2012., 2016., 2019.,
       2011., 1992., 2018., 2004., 2015., 2001., 2006., 1968., 2003.,
       2008., 2007., 2005., 1966., 2009., 1998., 2002., 1999., 2021.,
       1997., 1976., 1969., 1995., 1978., 1954., 1979., 1970., 1974.,
       1996., 1987., 2000., 1955., 1960., 1991., 1972., 1988., 1994.,
       1929., 1984., 1986., 1989., 1973., 1946., 1933., 1958., 1937.,
       1985., 1957., 1953., 1942., 1963., 1977., 1993., 1903., 1990.,
       1965., 1982., 1948., 1983., 1936., 1932., 1951., 1931., 1980.,
       1967., 1971., 1947., 1981., 1926., 1962., 1975., 1964., 1934.,
       1952., 1940., 1959., 1950., 1930., 1956., 1922., 1928., 2022.,
       1901., 1941., 1924., 1927., 1939., 1923., 1949., 1961., 1935.,
       1918., 1900., 1938., 1913., 1916., 1943., 1925., 1921., 1915.,
       1945., 1902., 1905., 1920., 1944., 1910., 1909.])

# Priprema podataka

In [18]:
dfz=df[['year','manufacturer','model','odometer','price']].dropna()[:10000]

In [19]:
from sklearn.model_selection import train_test_split

df_encoded = pd.get_dummies(dfz, columns=['manufacturer','model'])
y=df_encoded['price']
x=df_encoded.drop('price', axis=1)
train_df, test_df = train_test_split(x, test_size=0.2, random_state=42)
train_y, test_y = train_test_split(y, test_size=0.2, random_state=42)


In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(train_df, train_y)

# Make predictions on the test set
y_pred = rf_classifier.predict(test_df)

# Calculate accuracy
accuracy = accuracy_score(test_y, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print detailed classification report
print("\nClassification Report:")
print(classification_report(test_y, y_pred))

MemoryError: could not allocate 104923136 bytes

# Analiza

# Selekcija (vuk)

# Modeli mašinsko učenja

## Resampling (sara)

## F-regression (sara)

## Decision Tree (mixi)

## Random forest (mixi)

## Neuronske mreže (vuk)

# Zaključak (sara)

# Literatura (sara)

 ## https://chatgpt.com/

In [ ]:
pip freeze > requirements.txt